# converterTool
### This is a tool that converts and reduces the scraped JSON from TGCollector for use with the DistilBERT model 

 George Cotea, 2024

In [1]:
# Importing the necessary libraries
import json
import pandas as pd
import os

In [20]:
 path = '/Users/george/Desktop/Uni/2023-2024/Thesis/Final Thesis Collection/scrapedJSON'
#Change this to the path of the folder containing the JSON files 
csv_path = '/Users/george/Desktop/Uni/2023-2024/Thesis/Final Thesis Collection/testCSV'
#Change this to the path of the folder where you want the CSV files to be saved
for filename in os.listdir(path):
    if filename.endswith(".json"):
        with open(os.path.join(path, filename)) as f:
            data = json.load(f)
            df = pd.DataFrame(data)
            df = df[["id", "message"]]

In [21]:
# Importing regex for text preprocessing
import re
# Regex patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)

link_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

# Function to check if a string contains only emojis or links
def contains_only_emojis_or_links(text):
    cleaned_text = emoji_pattern.sub(r'', text)  # Remove emojis
    cleaned_text = link_pattern.sub(r'', cleaned_text)  # Remove links
    return not cleaned_text.strip()

# Function to remove emojis and links from a string
def remove_emojis_and_links(text):
    text = emoji_pattern.sub(r'', text)  # Remove emojis
    text = link_pattern.sub(r'', text)  # Remove links
    return text

# Remove rows that contain only emojis or links
df['remove'] = df['message'].apply(contains_only_emojis_or_links)
df = df[~df['remove']]

# Clean messages to remove emojis and links
df['message'] = df['message'].apply(remove_emojis_and_links)

# Drop the auxiliary 'remove' column
df.drop(columns=['remove'], inplace=True)


In [22]:
# Save df to a csv file
df.to_csv(os.path.join(csv_path, filename[:-5] + ".replace"), index=False)